In [8]:
import json
import requests
import time
import os

In [ ]:
OPENSEA_API_BASE = "https://api.opensea.io/api/v2"
OPENSEA_API_KEY = "" 
OPENSEA_API_BASE_V1 = "https://api.opensea.io/api/v1" 
collection_slug="doodles-official"

In [10]:
get_nfts_by_collection_url = "https://api.opensea.io/api/v2/collection/{collection_slug}/nfts?limit={limit}&next={next_string}"


In [11]:
def make_api_request(url, headers=None, params=None):
    """Make API request with basic rate limiting"""
    if headers is None:
        headers = {
            "Accept": "application/json",
            "X-API-KEY": OPENSEA_API_KEY
        }
    
    try:
        response = requests.get(url, headers=headers, params=params)
        
        # Handle rate limiting
        if response.status_code == 429:
            print("Rate limited, waiting 1 second...")
            time.sleep(1)
            return make_api_request(url, headers, params)
        
        # Raise exception for other errors
        response.raise_for_status()
        
        return response.json()
    except Exception as e:
        print(f"Error making request to {url}: {e}")
        return None

In [12]:
def get_nfts_by_collection(collection_slug, limit=200, next_string=""):
    """
    Get NFTs by collection slug
    Ouptut:
        {
    "nfts": [
        {
        "identifier": "string",
        "collection": "string",
        "contract": "string",
        "token_standard": "string",
        "name": "string",
        "description": "string",
        "image_url": "",
        "display_image_url": "",
        "display_animation_url": "",
        "metadata_url": "",
        "opensea_url": "",
        "updated_at": "string",
        "is_disabled": true,
        "is_nsfw": true
        }
    ],
    "next": "string"
    }
    """
    url = get_nfts_by_collection_url.format(collection_slug=collection_slug,limit=limit, next_string=next_string)
    return make_api_request(url)

In [13]:
# Get all NFTs in the collection
os.makedirs(f"./data/{collection_slug}", exist_ok=True)
nfts_output_file = f"./data/{collection_slug}/nfts_by_collection_data.json"
raw_output_file = f"./data/{collection_slug}/raw_response_data.json"

# Initialize or load existing nfts_json
try:
    with open(nfts_output_file, 'r') as f:
        nfts_json = json.load(f)
except FileNotFoundError:
    nfts_json = []

# Initialize or load existing raw responses
try:
    with open(raw_output_file, 'r') as f:
        raw_responses = json.load(f)
except FileNotFoundError:
    raw_responses = []


# Get first page of NFTs
nfts_dic = get_nfts_by_collection(collection_slug)
nfts_json += nfts_dic['nfts']  # Append to existing data
next_string = nfts_dic['next']

# Add the raw response to our list of responses
raw_responses.append(nfts_dic)

# Save first page immediately - both formats
with open(nfts_output_file, 'w') as f:
    json.dump(nfts_json, f, indent=4)
    
with open(raw_output_file, 'w') as f:
    json.dump(nfts_dic, f, indent=4)


# Get remaining pages
while next_string:
    nfts_dic = get_nfts_by_collection(collection_slug, next_string=next_string)
    nfts_json += nfts_dic['nfts']  # Append new NFTs
    raw_responses.append(nfts_dic)  # Append new raw response
    
    # Save both updated files
    with open(nfts_output_file, 'w') as f:
        json.dump(nfts_json, f, indent=4)
    
    with open(raw_output_file, 'w') as f:
        json.dump(raw_responses, f, indent=4)
    if 'next' in nfts_dic.keys():
        next_string = nfts_dic['next']
    else:
        next_string = None